In [1]:
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import transformations
import config
%matplotlib inline

In [2]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery('2,7,8,9')", conn)


starter_data = RideWaits.copy()

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)

C:\Users\Chrisrtopher\Documents\DisneyWaits\src\transformations.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\Chrisrtopher\Documents\DisneyWaits\src\transformations.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\Chrisrtopher\Documents\DisneyWaits\src\transformations.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [63]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop([column_name], axis = 1)
    return df

def get_shift(day, steps):
    previous_steps = {}
    for i in range(1,1+steps):
        current_steps = []
        test_day_current = day.reset_index()
        for index,row in test_day_current.iterrows():
            if index in list(range(i)):
                current_steps.append(0)
            else:
                current_steps.append(test_day_current.loc[index - i,'Wait'])
        
        name = "previous_step"+str(i)
        previous_steps[name] = current_steps
    
    for key,value in previous_steps.items():
        day[key] = value
        
    return day
    
    
def shift_data(ride_data, shift_range):
    new_data_frame = pd.DataFrame()
    distinct_rides = list(ride_data['RideId'].unique())
    for ride in distinct_rides:
        this_ride = ride_data[ride_data['RideId'] == ride]
        day_list = list(this_ride['Date'].unique())
        for day in day_list:
            day_data = this_ride[this_ride['Date'] == day]
            new_data = get_shift(day_data, shift_range)
            new_data_frame = pd.concat([new_data_frame, new_data])

    return new_data_frame

def model_transformation(data, num_shifts, start_day = True):
    ride_waits = data
    
    important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
    ride_waits = ride_waits[important_columns]
    ride_waits = ride_waits[ride_waits['Location'] != ""]
    
    ride_waits = ride_waits.dropna(how = "any")
    
    if start_day == False:
        ride_waits = shift_data(ride_waits,num_shifts)
    
    dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
    ride_waits = ride_waits.drop(['Name'], axis = 1)
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    correlation = ride_waits.corr()['Wait']
    key_correlations = correlation[abs(correlation) > .005]
    important_cols = list(key_correlations.index)
    shift_columns = []
    if start_day == False:
        shift_columns = ["previous_step" + str(x+1)  for x in range(num_shifts)]
    important_cols = important_cols + ["Wait","MinutesSinceOpen"] + shift_columns
    important_cols = [x for x in important_cols if x != "Weekend_0"]
    important_cols = list(set(important_cols))
    ride_waits_key = ride_waits[important_cols]
    
    
    return ride_waits_key



def new_data_transform(data, num_shifts, important_cols, start_day = True):
    ride_waits = data
    
    important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
    ride_waits = ride_waits[important_columns]
    ride_waits = ride_waits[ride_waits['Location'] != ""]
    
    ride_waits = ride_waits.dropna(how = "any")
    
    if start_day == False:
        ride_waits = shift_data(ride_waits,num_shifts)
    
    dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
    ride_waits = ride_waits.drop(['Name'], axis = 1)
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    missing_cols = [x for x in important_cols if x not in ride_waits.columns]
    for col in missing_cols:
        ride_waits[col] = 0
        
    return ride_waits

In [64]:
model_data = model_transformation(RideWaits, 3)

In [65]:
model_data.columns

Index(['RideId_80010129', 'RideId_80010173', 'RideId_80010150',
       'CharacterExperience_0', 'Location_Discovery Island',
       'Location_DinoLand U.S.A.', 'RideId_80010220', 'Month_10', 'EMHDay_0',
       'DayOfWeek_2',
       ...
       'Month_9', 'RideId_80010193', 'Location_Main Street, U.S.A.',
       'Location_Pandora   The World of Avatar', 'IntellectualProp_Pixar',
       'Location_Africa', 'RideId_207395', 'RideId_80010162', 'TimeSinceOpen',
       'RideId_80010190'],
      dtype='object', length=117)

In [66]:
best_params = {'criterion': 'mse',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

In [67]:
important_columns = [x for x in model_data.columns if x != "Wait"]

In [68]:
%%time
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(**best_params)
clf.fit(model_data[important_columns], model_data['Wait'])


Wall time: 44.8 s


In [69]:
#now we have a model, now we need to create a new data frame for values with the columns listed above for prediction

In [70]:
rides = list(set(starter_data['Name']))
current_ride = starter_data[starter_data['Name'] == rides[23]]

In [71]:
ride_starter = current_ride.iloc[[0]]

In [72]:
rides

['Meet Chewbacca at Star Wars Launch Bay',
 'Meet Ariel at Her Grotto',
 'its a small world',
 'Star Tours- The Adventures Continue',
 'TriceraTop Spin',
 'DINOSAUR',
 'Star Wars Launch Bay: Encounter Kylo Ren',
 'Mad Tea Party',
 'Mickeys PhilharMagic',
 'Prince Charming Regal Carrousel',
 'Tom Sawyer Island',
 'Splash Mountain',
 'Its Tough to be a Bug!',
 'Living with the Land',
 'Avatar Flight of Passage',
 'The Magic Carpets of Aladdin',
 'Meet Disney Pals at the Epcot Character Spot',
 'Meet Tinker Bell at Town Square Theater',
 'Walt Disneys Enchanted Tiki Room',
 'Journey Into Imagination With Figment',
 'Mickey and Minnie Starring in Red Carpet Dreams',
 'The Twilight Zone Tower of Terror',
 'Swiss Family Treehouse',
 'Toy Story Mania!',
 'Buzz Lightyears Space Ranger Spin',
 'Kilimanjaro Safaris',
 'Soarin',
 'Royal Sommerhus',
 'A Pirates Adventure ~ Treasures of the Seven Seas',
 'Spaceship Earth',
 'Tomorrowland Speedway',
 'Peter Pans Flight',
 'Frozen Ever After',
 'Rock

In [73]:
from datetime import datetime
from pytz import timezone

In [74]:
tz = timezone('US/Eastern')

In [75]:
dtime = datetime.now(tz)
dtime = dtime.replace(hour = 7,minute = 0, second = 0, microsecond = 0)
date = dtime.date()
time = dtime.time().strftime("%H:%M")

In [76]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

In [77]:
end_time = dtime.replace(hour = 23, minute = 45, second = 0, microsecond = 0)

In [78]:
time_list = date_range(dtime, end_time, 15, 'minutes')

In [79]:
time_list = [x.time().strftime("%H:%M") for x in time_list]

In [84]:

predictions_frame = pd.DataFrame()
for time in time_list:
    ride_starter['Time'] = time
    row_data = ride_starter.copy()
    try:
        row_data = transformations.transformData(row_data)
    except:
        continue
    #print(row_data)
    predictions_frame = pd.concat([predictions_frame,row_data])
    print(predictions_frame.iloc[[predictions_frame.shape[0]-1]])
    model_predictions_frame = new_data_transform(predictions_frame,3, important_columns)
    predictions_frame['Wait'] = clf.predict(model_predictions_frame[important_columns])
    #print(predictions_frame)



C:\Users\Chrisrtopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  09:15:00    20  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay TimeSinceOpen  \
2812                   0     0          1       0             0   

     TimeSinceMidday  MagicHourType  MinutesSinceOpen  Month  \
2812               5           None              15.0      9   

      TimeSinceRideOpen  
2812               3763  

[1 rows x 27 columns]
      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  09:30:00  20.0  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay Tim

      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  12:45:00  20.0  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay TimeSinceOpen  \
2812                   0     0          1       0             3   

     TimeSinceMidday  MagicHourType  MinutesSinceOpen  Month  \
2812               2           None             225.0      9   

      TimeSinceRideOpen  
2812               3763  

[1 rows x 27 columns]
      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  13:00:00  20.0  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay Tim

      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  16:15:00  20.0  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay TimeSinceOpen  \
2812                   0     0          1       0             7   

     TimeSinceMidday  MagicHourType  MinutesSinceOpen  Month  \
2812               2           None             435.0      9   

      TimeSinceRideOpen  
2812               3763  

[1 rows x 27 columns]
      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  16:30:00  20.0  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay Tim

      RideId       Date      Time  Wait              Name OpeningDate  \
2812  209857 2018-09-19  19:45:00  20.0  Toy Story Mania!  2008-05-31   

           Tier        Location IntellectualProp ParkId        ...          \
2812  headliner  Toy Story Land            Pixar      8        ...           

     CharacterExperience inEMH  validTime  EMHDay TimeSinceOpen  \
2812                   0     0          1       0            10   

     TimeSinceMidday  MagicHourType  MinutesSinceOpen  Month  \
2812               5           None             645.0      9   

      TimeSinceRideOpen  
2812               3763  

[1 rows x 27 columns]


In [81]:
predictions_frame

,RideId,Date,Time,Wait,Name,OpeningDate,Tier,Location,IntellectualProp,ParkId,...,CharacterExperience,inEMH,validTime,EMHDay,TimeSinceOpen,TimeSinceMidday,MagicHourType,MinutesSinceOpen,Month,TimeSinceRideOpen
2812,209857,2018-09-19,09:15:00,43.580726,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,0,5,None,15.0,9,3763
2812,209857,2018-09-19,09:30:00,43.580726,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,0,5,None,30.0,9,3763
2812,209857,2018-09-19,09:45:00,43.580726,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,0,5,None,45.0,9,3763
2812,209857,2018-09-19,10:00:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,1,4,None,60.0,9,3763
2812,209857,2018-09-19,10:15:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,1,4,None,75.0,9,3763
2812,209857,2018-09-19,10:30:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,1,4,None,90.0,9,3763
2812,209857,2018-09-19,10:45:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,1,4,None,105.0,9,3763
2812,209857,2018-09-19,11:00:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,2,3,None,120.0,9,3763
2812,209857,2018-09-19,11:15:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,2,3,None,135.0,9,3763
2812,209857,2018-09-19,11:30:00,44.045049,Toy Story Mania!,2008-05-31,headliner,Toy Story Land,Pixar,8,...,0,0,1,0,2,3,None,150.0,9,3763


In [ ]:
#perhaps for first of day predictions do not incorporate a shifting feature

In [82]:
important_columns

['RideId_80010129',
 'RideId_80010173',
 'RideId_80010150',
 'CharacterExperience_0',
 'Location_Discovery Island',
 'Location_DinoLand U.S.A.',
 'RideId_80010220',
 'Month_10',
 'EMHDay_0',
 'DayOfWeek_2',
 'RideId_80010177',
 'EMHDay_1',
 'ParkId_2',
 'Weekend_1',
 'RideId_80010208',
 'RideId_80010228',
 'RideId_80010191',
 'RideId_136550',
 'DayOfWeek_6',
 'RideId_17564219',
 'Location_Commissary Lane',
 'RideId_18189394',
 'Tier_minor_attraction',
 'DayOfWeek_3',
 'Location_Echo Lake',
 'IntellectualProp_Animated',
 'CharacterExperience_1',
 'RideId_62992',
 'IntellectualProp_None',
 'RideId_15850196',
 'RideId_80010178',
 'IntellectualProp_Pirates',
 'TimeSinceMidday',
 'IntellectualProp_Avatar',
 'Location_Grand Avenue',
 'Location_Tomorrowland',
 'RideId_18375495',
 'RideId_80010196',
 'RideId_16124144',
 'RideId_80010153',
 'RideId_18904138',
 'RideId_80010199',
 'Location_Future World',
 'IntellectualProp_StarWars',
 'inEMH_1',
 'RideId_80010123',
 'IntellectualProp_Muppets',
